This notebook is used to process original taxi trip data to count data.   
At the end of project it will be merged into the main notebook.


In [ ]:
# import libraries to process parquet files and geospatial data
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# create a function to read and process parquet files from folder and return a dataframe
def read_and_count_parquet(folder_path):
    '''
    INPUT: folder_path - path to folder containing parquet files
    OUTPUT: saves the processed dataframe to csv file
    '''
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".parquet"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_parquet(file_path)

            # rename columns for yellow taxi data
            if filename.split('_')[0] == 'yellow':
                df = df.rename(columns={'tpep_pickup_datetime':'lpep_pickup_datetime',
                                                           'tpep_dropoff_datetime':'lpep_dropoff_datetime'})
            df = process_data(df)  # process the dataframe
            try:
                save_path = f'../data/processed_nyc_data/{filename.split(".")[0]}.csv'
            except:
                print('save_path is not defined')
            save_path = save_path.replace('trip', 'count')
            df.to_csv(save_path, index=False)  # save the dataframe to csv file
            print(f'{filename} is processed')
            
    return 'Done!'

# create a function to process the dataframe and return a counted dataframe
def process_data(df):
    '''
    INPUT: df - dataframe of parquet files
    OUTPUT: df - counted dataframe of parquet files
    '''
    # select columns that are needed
    df = df[['lpep_pickup_datetime', 'passenger_count', 
              'PULocationID', 'DOLocationID', 'tip_amount', 'total_amount']]

    # delete rows with values >263 in 'PULocationID' and 'DOLocationID' columns
    df = df[(df['PULocationID'] <= 263) & (df['DOLocationID'] <= 263)]
    # fill missing values in 'passenger_count' column with 1
    df['passenger_count'] = df['passenger_count'].fillna(1)
    # dropna in 'total_amount' and 'tip_amount' columns
    df = df.dropna(subset=['total_amount','tip_amount'])
    # calculate the trip_fee = Total_amount - Tip_amount
    df['trip_fee'] = df['total_amount'] - df['tip_amount']


    # delete columns that are not needed anymore
    df = df.drop(columns=['total_amount','tip_amount'])
    # delete rows with negative values and 0 in 'trip_fee' column
    df = df[df['trip_fee'] > 0]

    # count the number of trips in each day, hour, PULocationID and DOLocationID
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['day'] = df['lpep_pickup_datetime'].dt.day
    df['hour'] = df['lpep_pickup_datetime'].dt.hour
    count_df = df.groupby(['day','hour','PULocationID','DOLocationID']).count().reset_index()
    
    # calculate the average trip_fee in each day, hour, PULocationID and DOLocationID
    count_df['trip_fee'] = df.groupby(['day','hour',
                                       'PULocationID','DOLocationID']).mean().reset_index()['trip_fee']

    # rename the column 'lpep_pickup_datetime' to 'trip_count'
    count_df = count_df.rename(columns={'lpep_pickup_datetime':'trip_count'})

    # calculate the passenger_cofficient = passenger_count / trip_count
    # passenger_cofficient is the average number of passengers in each trip

    count_df['passenger_cofficient'] = df.groupby(['day','hour',
                                                    'PULocationID','DOLocationID']).mean().reset_index()['passenger_count']
     
    
    return count_df

useful columns: 
* lpep _pickup_datetime
* passenger_count
* PULocation
* DOLocation
* Tip_amount
* Total_amount

for:
* passenger_cofficient = passenger_count / trip_count
* trip_fee = Total_amount - Tip_amount



In [ ]:
# process parquet files from folder
folder_path = '../data/NYC_taxi_data/'
read_and_count_parquet(folder_path)

In [ ]:
# merge the green and yellow taxi data
for i in range(2017,2022):
    df_green = pd.read_csv(f'../data/processed_nyc_data/green_countdata_{i}-12.csv')
    df_yellow = pd.read_csv(f'../data/processed_nyc_data/yellow_countdata_{i}-12.csv')
    
    # merge the green and yellow taxi data by day, hour, PULocationID and DOLocationID
    # which trip_count = yellow + green
    # passenger_count = yellow + green
    # trip_fee = (yellow * yellow_trip_fee + green * green_trip_fee) / (yellow + green)
    # passenger_cofficient = (yellow * yellow_passenger_cofficient + green * green_passenger_cofficient) / (yellow + green)
    df = pd.merge(df_green, df_yellow, how='outer', on=['day','hour','PULocationID','DOLocationID'])
    df = df.fillna(0)
    df['trip_count'] = df['trip_count_x'] + df['trip_count_y']
    df['passenger_count'] = df['passenger_count_x'] + df['passenger_count_y']
    df['trip_fee'] = round((df['trip_fee_x'] * df['trip_count_x'] + df['trip_fee_y'] * df['trip_count_y']) / df['trip_count'],2)  # round 2
    df['passenger_cofficient'] = round((df['passenger_cofficient_x'] * df['trip_count_x'] + df['passenger_cofficient_y'] * df['trip_count_y']) / df['trip_count'], 2)  # round 2
    df = df.drop(columns=['trip_count_x','trip_count_y','passenger_count_x','passenger_count_y',
                            'trip_fee_x','trip_fee_y','passenger_cofficient_x','passenger_cofficient_y'])
    
    # save the dataframe to csv file
    df.to_csv(f'../data/processed_nyc_data/countdata_{i}-12.csv', index=False)
    print(f'countdata_{i}-12.csv is saved')
    